[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RlNZLER/Subtitle-Generator-Translator/blob/main/src/whisper.ipynb)

# Step 1:
# Execute the following task and upload an audio file, or files, to the content directory while you wait for the task to complete.

In [ ]:
!pip install git+https://github.com/openai/whisper.git

# Step 2:
# Once the above task has completed and all audio files have successfully been uploaded to the content directory, execute the follow task.

In [ ]:
import glob, os
audioFiles = glob.glob('/content/*.*')
for i in range(len(audioFiles)):
  for audioFile in glob.glob(audioFiles[i]):
    baseFile = os.path.splitext(audioFile)[0]
    !whisper --model turbo --output_format srt --task transcribe "$audioFile"
    os.remove(audioFile)

# Step 3:
# Download your SRT files from the content directory.